<a href="https://colab.research.google.com/github/snow-white2024/first-repository/blob/master/Chart_Journeys.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# 사용할 라이브러리 불러오기
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
import numpy as np

In [ ]:
country = input('검색하고 싶은 도시를 입력하세요 : ')
price = int(input('1박에 사용할수있는 금액을 입력하세요(달러) : '))

검색하고 싶은 도시를 입력하세요 : Belize
1박에 사용할수있는 금액을 입력하세요(달러) : 100


In [ ]:
airbnb = f"/content/drive/MyDrive/airbnb/{country}_listings.csv"
original_data = pd.read_csv(airbnb)

In [ ]:
#rating변수에 rating에서 사용할 데이터만 저장
rating = ['id', 'host_response_time', 'host_response_rate', 'host_identity_verified', 'bathrooms', 'bedrooms', 'amenities', 'review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness', 'review_scores_communication', 'review_scores_location', 'review_scores_value']
display = ['id', 'name', 'price', 'listing_url']
stay_rating = original_data[rating]
display_price = original_data[display]

In [ ]:
#stay_rating에서 null값을 0으로 치환
stay_rating = stay_rating.fillna(0)

In [ ]:
#host_response_time컬럼에서 within a few hours, within an hour, a few days or more, within a day를 제외한 이상치 데이터 제거
outliers =  (stay_rating['host_response_time']== 0) | (stay_rating['host_response_time'] == 'within a few hours') | (stay_rating['host_response_time'] == 'within an hour') | (stay_rating['host_response_time'] == 'a few days or more') | (stay_rating['host_response_time'] == 'within a day')

stay_rating = stay_rating[outliers]

In [ ]:
#host_response_rate컬럼에서 %를 제거하고 float형식으로 변환 및 Null값을 0으로 치환
stay_rating['host_response_rate'] = stay_rating['host_response_rate'].str.rstrip('%').astype(float)
stay_rating['host_response_rate'] = stay_rating['host_response_rate'].fillna(0)

In [ ]:
#host_identity_verified컬럼에서 t와 f가 아닌 이상치 제거(0은 Null값 채우는 용)
outliers =  (stay_rating['host_identity_verified']=='t') | (stay_rating['host_identity_verified'] == 'f') | (stay_rating['host_identity_verified']== 0)

stay_rating = stay_rating[outliers]

In [ ]:
#ast라이브러리에서 literal_eval불러오기
from ast import literal_eval

#amenities컬럼을 list형태로 변환
stay_rating['amenities'] = stay_rating['amenities'].apply(literal_eval)

In [ ]:
#review_scores_rating칼럼에 5가 넘는 이상치를 제거
outliers = (stay_rating['review_scores_rating'] <= 5)

stay_rating = stay_rating[outliers]

In [ ]:
#review_scores_accuracy칼럼에 5가 넘는 이상치를 제거
outliers = (stay_rating['review_scores_accuracy'] <= 5)

stay_rating = stay_rating[outliers]

In [ ]:
#review_scores_cleanliness칼럼에 5가 넘는 이상치를 제거
outliers = (stay_rating['review_scores_cleanliness'] <= 5)

stay_rating = stay_rating[outliers]

In [ ]:
#review_scores_communication칼럼에 5가 넘는 이상치를 제거
outliers = (stay_rating['review_scores_communication'] <= 5)

stay_rating = stay_rating[outliers]

In [ ]:
#review_scores_location칼럼에 5가 넘는 이상치를 제거
outliers = (stay_rating['review_scores_location'] <= 5)

stay_rating = stay_rating[outliers]

In [ ]:
#review_scores_value칼럼에 5가 넘는 이상치를 제거
outliers = (stay_rating['review_scores_value'] <= 5)

stay_rating = stay_rating[outliers]

In [ ]:
#stay_rating에 chart_rating칼럼 추가
stay_rating['chart_rating'] = 0

In [ ]:
#host_response_time에 within an hour면 0.3점, within a few hours면 0.2점, within a day면 0.1점 나머지 값은 0점을 chart_rating에 추가한다.
conditions = [
    stay_rating['host_response_time'] == "within a day",
    stay_rating['host_response_time'] == "within a few hours",
    stay_rating['host_response_time'] == "within an hour"
]
values = [0.1, 0.2, 0.3]

stay_rating['chart_rating'] += np.select(conditions, values, default=0)

In [ ]:
#host_response_rate가 100%면 0.2점 40%미만이면 0점 40%~99%면 0.1점을chart_rating에 추가
conditions = [
    stay_rating['host_response_rate'] == 100,
    stay_rating['host_response_rate'] < 40
]
values = [0.2, 0]

stay_rating['chart_rating'] += np.select(conditions, values, default=0.1)

In [ ]:
#host_identity_verified(호스트 인증)이 되어있으면 0.5점을 chart_rating에 점수 추가
conditions = [
    stay_rating['host_identity_verified'] == 't',
    stay_rating['host_identity_verified'] == 'f'
]
values = [1, 0]

stay_rating['chart_rating'] += np.select(conditions, values, default=0)

In [ ]:
#bathrooms의 개수가 0개면 0점, 1개면 1점 1개 이상이면 1.2점을 chart_rating에 점수 추가
conditions = [
    stay_rating['bathrooms'] == 0,
    stay_rating['bathrooms'] == 1
]
values = [0, 1]

stay_rating['chart_rating'] += np.select(conditions, values, default=1.2)

In [ ]:
#bedrooms의 개수가 0개면 0점, 1개면 1점 1개 이상이면 1.2점을 chart_rating에 점수 추가
conditions = [
    stay_rating['bedrooms'] == 0,
    stay_rating['bedrooms'] == 1
]
values = [0, 1]

stay_rating['chart_rating'] += np.select(conditions, values, default=1.2)

In [ ]:
#amenities에 있는 편의시설 1개당 0.05점씩 최대 1점을 chart_rating에 점수 추가
stay_rating['chart_rating'] += stay_rating['amenities'].str.len().apply(lambda x: 1 if x >= 20 else x / 20)

In [ ]:
#review_scores_rating점수에 따라 chart_rating에 점수 추가
stay_rating['chart_rating'] += stay_rating['review_scores_rating'].apply(lambda x : x/5)

In [ ]:
#review_scores_accuracy점수에 따라 chart_rating에 점수 추가
stay_rating['chart_rating'] += stay_rating['review_scores_accuracy'].apply(lambda x : x/5/5)

In [ ]:
#review_scores_cleanliness점수에 따라 chart_rating에 점수 추가
stay_rating['chart_rating'] += stay_rating['review_scores_cleanliness'].apply(lambda x : x/5)

In [ ]:
#review_scores_communication점수에 따라 chart_rating에 점수 추가
stay_rating['chart_rating'] += stay_rating['review_scores_communication'].apply(lambda x : x/5/2)

In [ ]:
#review_scores_location점수에 따라 chart_rating에 점수 추가
stay_rating['chart_rating'] += stay_rating['review_scores_location'].apply(lambda x : x/5*1.2)

In [ ]:
#review_scores_value점수에 따라 chart_rating에 점수 추가
stay_rating['chart_rating'] += stay_rating['review_scores_value'].apply(lambda x : x/5*1.2)

In [ ]:
#display_price에 stay_rating에서 만든 chart_rating복사하여 붙이기
display_price['chart_rating'] = stay_rating['chart_rating']

<ipython-input-64-4738a7a6156c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  display_price['chart_rating'] = stay_rating['chart_rating']


In [ ]:
#chart_rating컬럼을 str형식에서 특수문자를 없에고 float형식으로 바꾼다
display_price['price'] = display_price['price'].str.replace('[^\d.]', '', regex=True).astype(float)
display_price['price'] = display_price['price'].fillna(0)

<ipython-input-65-19453f34860a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  display_price['price'] = display_price['price'].str.replace('[^\d.]', '', regex=True).astype(float)
<ipython-input-65-19453f34860a>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  display_price['price'] = display_price['price'].fillna(0)


In [ ]:
#display_price에서 사용자가 입력한 금액보다 큰 금액이 필요한 것들 지우기
outliers_price = (display_price['price'] < price)
display_price = display_price[outliers_price]
display_price.sort_values('price')

,id,name,price,listing_url,chart_rating
1260,39977442,"Gorgeous 3 bedroom house - ""Casa de"" -MB",0.0,https://www.airbnb.com/rooms/39977442,2.9000
1412,43115290,Come reconnect with nature,0.0,https://www.airbnb.com/rooms/43115290,0.4500
1411,43052908,Casita Naranja - Beauty and the Beach,0.0,https://www.airbnb.com/rooms/43052908,2.4500
1410,43052749,CASITA VERDE - Beachfront with a view of the Reef,0.0,https://www.airbnb.com/rooms/43052749,2.6500
1409,43044433,Casita Azul - Elegance on the Beach,0.0,https://www.airbnb.com/rooms/43044433,2.6500
...,...,...,...,...,...
2388,829117008865492741,Sands Coral Suite,99.0,https://www.airbnb.com/rooms/829117008865492741,9.2500
23,1135021,"Bay Breezes apt #3, Caye Caulker",99.0,https://www.airbnb.com/rooms/1135021,8.8732
22,1134718,"Bay Breezes apt #1, Caye Caulker",99.0,https://www.airbnb.com/rooms/1134718,9.0862
2065,657798707644105031,Pilgrim’s Paradise Cabin 3!,99.0,https://www.airbnb.com/rooms/657798707644105031,7.9684


In [ ]:
#사용자에게 중요한 정보 보여주기
draw_map = display_price[['id', 'chart_rating', 'listing_url']].sort_values('chart_rating', ascending=False).head(10)
display_price.sort_values('chart_rating', ascending=False).head(10)

,id,chart_rating,listing_url
341,16407477,9.8000,https://www.airbnb.com/rooms/16407477
408,17859805,9.7148,https://www.airbnb.com/rooms/17859805
178,9408664,9.6964,https://www.airbnb.com/rooms/9408664
1825,53255234,9.6720,https://www.airbnb.com/rooms/53255234
1283,40352367,9.6344,https://www.airbnb.com/rooms/40352367
410,17918037,9.6000,https://www.airbnb.com/rooms/17918037
742,25871729,9.6000,https://www.airbnb.com/rooms/25871729
3183,1212523864950432767,9.6000,https://www.airbnb.com/rooms/1212523864950432767
578,21693610,9.5722,https://www.airbnb.com/rooms/21693610
1275,40172424,9.5674,https://www.airbnb.com/rooms/40172424


In [ ]:
from IPython.display import display, HTML
from folium.plugins import MarkerCluster
import folium

HTML('<style>.output {flex-direction: row;}</style>')

geo_display= ['latitude','longitude', 'name']

test = draw_map
test[['latitude','longitude', 'name']] = original_data[['latitude','longitude', 'name']]

latitude = test['latitude'].mean()
longitude= test['longitude'].mean()

rating_map= folium.Map(location=[latitude, longitude], zoom_start=9)

rating_Marker = MarkerCluster().add_to(rating_map)

def getcolor(rating) :
  if rating >= 9:
    return 'red'
  elif rating >= 7:
    return 'blue'
  elif rating >= 5:
    return 'green'
  else :
    return 'yellow'

for _, row in test.iterrows():

    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=f"{row['name']} - url: {row['listing_url']}",
        tooltip=row['name'],
        icon=folium.Icon(color=getcolor(row['chart_rating']))
    ).add_to(rating_map)

display(display_price.sort_values('chart_rating', ascending=False).head(10))
rating_map

,id,name,price,listing_url,chart_rating
341,16407477,2 Bedroom Apt on 84 Acres near Spanish Lookout,99.0,https://www.airbnb.com/rooms/16407477,9.8000
408,17859805,Iguana Roost Great Base For Exploring Inland BZ,92.0,https://www.airbnb.com/rooms/17859805,9.7148
178,9408664,Close to town but in the country,57.0,https://www.airbnb.com/rooms/9408664,9.6964
1825,53255234,"Liana Glade Chalet, 2-bedroom, fully furnished",77.0,https://www.airbnb.com/rooms/53255234,9.6720
1283,40352367,The Joy of Life -Lamanai,91.0,https://www.airbnb.com/rooms/40352367,9.6344
410,17918037,Don Santiago Guest House Downtown ROOM 1,88.0,https://www.airbnb.com/rooms/17918037,9.6000
742,25871729,Conerstone Estate Vacation Wooden TiPi,81.0,https://www.airbnb.com/rooms/25871729,9.6000
3183,1212523864950432767,"Cute new home, great location, crazy view in Cayo",92.0,https://www.airbnb.com/rooms/1212523864950432767,9.6000
578,21693610,Dorm Bed in San Ignacio,34.0,https://www.airbnb.com/rooms/21693610,9.5722
1275,40172424,Hostel Bed in San Pedro Town,49.0,https://www.airbnb.com/rooms/40172424,9.5674
